In [1]:
import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'retina'
plt.style.use('ggplot')
import pandas as pd
import numpy as np

import seaborn as sns
sns.set()

from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.model_selection import GridSearchCV, cross_val_score

from sklearn.metrics import accuracy_score

from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import learning_curve

In [2]:
# load the data
df = pd.read_csv('./Data/Final_18_features_extracted_V2.csv')

In [3]:
X = df.drop(columns=["Region","File Name"],axis=1)
y = df['Region']

- Selecting first 7 most important features for X

In [6]:
X = df[['AMP_ENV','MFCC1','SPEC_BAND','MFCC6','MFCC7','MFCC3','MFCC7']]

In [8]:
X.shape,y.shape

((9059, 7), (9059,))

In [9]:
X_train,X_test,y_train,y_test = train_test_split(
    X,y,test_size=0.3,random_state=17,shuffle=True
)

In [10]:
skf = StratifiedKFold(n_splits=5,shuffle=True,random_state=17)

rfc = RandomForestClassifier(random_state=17,n_jobs=-1)

In [11]:
results = cross_val_score(rfc,X_train,y_train,cv=skf)
results.mean()*100

62.86056981204307

In [12]:
rfc.fit(X_train,y_train)

RandomForestClassifier(n_jobs=-1, random_state=17)

In [13]:
accuracy_score(rfc.predict(X_train),y_train)

1.0

In [14]:
forest_params = {
    'n_estimators':[10,15,20,25],
    'max_features': [4],
    'min_samples_leaf': [16,18,20],
    'max_depth': [7,10,13],
    'min_samples_split':[20,25,30]

}
rfc = RandomForestClassifier(n_estimators=50,random_state=17,n_jobs=-1)

gcv = GridSearchCV(rfc,forest_params,n_jobs=-1,cv=skf,verbose=1)

In [15]:
gcv.fit(X_train,y_train)

Fitting 5 folds for each of 108 candidates, totalling 540 fits


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=17, shuffle=True),
             estimator=RandomForestClassifier(n_estimators=50, n_jobs=-1,
                                              random_state=17),
             n_jobs=-1,
             param_grid={'max_depth': [7, 10, 13], 'max_features': [4],
                         'min_samples_leaf': [16, 18, 20],
                         'min_samples_split': [20, 25, 30],
                         'n_estimators': [10, 15, 20, 25]},
             verbose=1)

In [17]:
gcv.best_score_

0.5926496434013717

In [18]:
accuracy_score(gcv.best_estimator_.predict(X_train),y_train)

0.6778110708090207